In [2]:
import numpy as np
import os
import torch
import uproot
from torch.utils.data import Dataset

/home/zhou907/.conda/envs/cent7/5.1.0-py36/GANS2/lib/python3.6/site-packages/uproot/reading.py:186: FutureWarning: XRootD 5.1.1 is not fully supported; either upgrade to 5.2.0+ or set

    open.defaults["xrootd_handler"] = uproot.MultithreadedXRootDSource

  warnings.warn(message, FutureWarning)


In [51]:
class ProductionModeDataset(Dataset):
    """
    This class will load the minitree and create the dataset loaded into the nns
    
    0) load the minitree properly
    
    1) remove 0 and 2 type events so we have the same number as qqbar
    
    2) normalize all data
    
    3) ensure this is stored in a numpy array with the correct arrangement of data and a corresponding list of datatypes
    
    4) split this up into training and evaluating datasets
    
    Args:
        :param root (string): is root directory and file name of minitree
        :param split (boolean): tells whether to split into training and eval
        :param normalize (boolean): tells whether to normalize data
        :param remove (boolean): tells whether we should remove excess data for non-qqbar events or duplicate qqbar
        :param train (boolean): tells whether we are training or evaluating
    """
    
    def __init__(self, root, split=True, normalize=True, remove=True, train=True):
        self.events = uproot.open(root)
        self.training = train
        #self.events = self.events['Events']
        
        #data_list = data = uproot.open(depo_dir + root_f, key = "ttBar_treeVariables_step8;3")
        
        
        # TODO: make less complex when not needed later
#         data_list = [x in self.events["ttBar_treeVariables_step8;3"].keys() if ((("phi" in x) or ("eta" in x) \
#                                                                              or ("pt" in x) or ("production" in x)) and \
#                                                                              ("gen" not in x))]
#         print(str(data_list))
        print(self.events.keys())
        data_list = self.events["ttBar_treeVariables_step8;4;1"].keys()
    
        self.events_array = np.array([self.events["ttBar_treeVariables_step8;4;1" + "/" + k].array(library="np") for k in data_list])
        print(len(data_list))
    
        self.events_array = np.transpose(self.events_array)   # turn to columns of data
        
        if remove:
            """
            Here is where we remove the excess data
            """
            # find index of production_mode:
            index = 0
            for i in range(len(data_list)):
                if data_list[i] == "production_mode":
                    index = i
                    break
                    
            self.events_array = self.events_array[np.argsort(self.events_array[:, index])]            
            
            # find first and last index with production mode 1
            first = 0
            last = 0
            found_first = False
            
            for i in range(len(self.events_array[:,index])):
                if not found_first:
                    if self.events_array[i ,index] == 1:
                        first = i
                        found_first = True
                
                if self.events_array[i,index] == 2:
                    last = i - 1
                    break
                    
            num_qqbar = last + 1 - first
            print("num qqbar = " + str(num_qqbar))
            # remove the extra gg and other
            max_len = len(self.events_array[:,0])
            self.events_array = np.delete(self.events_array, list(range(num_qqbar, first)) + \
                                          list(range(max_len - (max_len - (last + 1) -num_qqbar), max_len)), 0)
            
        # normalize here
            #print(len(self.events_array[:,0]))
            #columns = self.events_array[1]
            
            #min_a = []
            #max_a = []
            #range_a = []
            #update_a = []
            
            for i in range(len(self.events_array[0,:])-3):#the last three is subtracted, thust not there any more
                ori_a = self.events_array[0,i]#original number
                #print('ori_a =',ori_a)
                min_a = np.min(self.events_array[:,i])#min
                max_a = np.max(self.events_array[:,i])#max
                range_a = max_a - min_a#range
                #print('range =',range_a)
                norm_a = (ori_a - min_a) / (range_a) #normalized list. i dont know what to do with it.
                print('normalized =', norm_a)
                
            #range_a.append(max_a - min_a)
            
            #print(min_a)
        
        # split here        
        if split:
            # shuffle so no longer sorted before splitting
            np.random.shuffle(self.events_array)
            
            train_size = int(len(self.events_array[:,0])*81/100)
#             eval_size = len(self.events_array[:,0]) - train_size
            
            self.train_array, self.eval_array = np.split(self.events_array, [train_size])
            print("training " + str(self.train_array.shape))
            print("evaluating " + str(self.eval_array.shape))
            
        else:
            self.train_array = self.events_array
            self.eval_array = self.events_array
                
    def __getitem__(self, index):
        if self.training:
            return self.train_array[index]
        return self.eval_array

In [52]:
root_path = "/depot-new/cms/top/mcnama20/TopSpinCorr-Run2-Entanglement/CMSSW_10_2_22/src/TopAnalysis/Configuration/analysis/diLeptonic/three_files"

file = root_path + "/ee_modified_root.root"

ProductionModeDataset(file)

['ttBar_treeVariables_step8;4;1']
95
num qqbar = 6324
normalized = 0.04181489141865737
normalized = 0.11650462404529133
normalized = 0.22412155890396215
normalized = 0.22410214713207907
normalized = 0.12467095565544818
normalized = 0.04889804212858872
normalized = 0.18488920977908835
normalized = 0.06902074585196209
normalized = 0.2338219584895072
normalized = 0.13764307235102602
normalized = 0.16761036866299922
normalized = 0.13834151267795958
normalized = 0.9351173300708409
normalized = 0.2275607615231607
normalized = 0.5941679603374599
normalized = 0.83922264286782
normalized = 0.3025786569183435
normalized = 0.3945229366414417
normalized = 0.03445851468679508
normalized = 0.22283083544352061
normalized = 0.33674537260656295
normalized = 0.0834481290465456
normalized = 0.3568475029528626
normalized = 0.6128022712263326
normalized = 0.1325197294862518
normalized = 0.10321745556763842
normalized = 0.11311036477897284
normalized = 0.09205083640260057
normalized = 0.31613433791083856
no

/home/zhou907/.conda/envs/cent7/5.1.0-py36/GANS2/lib/python3.6/site-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in double_scalars


In [90]:
x = np.array(([1,2,3],[34,5,6], [4,5,7]))
x

array([[ 1,  2,  3],
       [34,  5,  6],
       [ 4,  5,  7]])

In [127]:
np.delete(x, [1,2],0)

array([[1, 2, 3]])

In [134]:
l = [1,2,3]
k = [4,5]

In [135]:
l+k

[1, 2, 3, 4, 5]